In [13]:
import math
import subprocess


def extract_audio_from_video(video_path, audio_path):
    command = f"ffmpeg -i {video_path} -vn {audio_path}"
    subprocess.run(command, shell=True)
    
def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = 1000 * 60 * chunk_size
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * ten_minutes
        end_time = (i + 1) * ten_minutes
        
        chunk = track[start_time:end_time]
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")
    
# extract_audio_from_video("files/worlds_semi_final.mp4", "files/audio.mp3")
cut_audio_in_chunks("files/audio.mp3", 10, "files/chunks")

In [8]:
from pydub import AudioSegment


track = AudioSegment.from_mp3("files/audio.mp3")
ten_minutes = 10 * 60 * 1000
track.duration_seconds

4454.841179138322

In [14]:
import openai
import glob


def transcribe_chunks(chunk_folder, destination):
    files = sorted(glob.glob(f"{chunk_folder}/*.mp3"))
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file,
                response_format="text",
                language="ko"
            )
            # final_transcript += transcript
            text_file.write(transcript)
    # with open(destination, "w") as f:
    #     f.write(final_transcript)

transcribe_chunks("files/chunks", "files/transcript.txt")